In [1]:
import numpy as np 
import random
import matplotlib.pyplot as plt
from scipy.stats import uniform 
from scipy.stats import norm
import os, time
import pandas as pd
import csv
import matplotlib.colors as colors 

## Functions

In [2]:
def rounding(frqs):
    """
    The input to this function is frqs, a list of frequencies that all must
    be between 0 and 1 and sum to 1. The output is the list of frequencies
    without rounding error.
    """
    frqs_in_bounds = []
    for frq in frqs:  # Ensure that all frequencies are in [0,1]
        if frq < 0:
            frq = 0
        elif frq > 1:
            frq = 1
        frqs_in_bounds.append(frq)

    normalizer = sum(frqs_in_bounds)
    return [frq / normalizer for frq in frqs_in_bounds] # Ensure that all frequencies sum to 1 

In [3]:
def get_angle(a, b):
    """
    This function returns the smallest angle between points a and b, in
    radians, assuming that points a and b lie on a circle that has 
    circumference 1. 
    """
    r = 1 / (2 * np.pi) # Radius = circumference / 2π 
    dist = abs(a-b) # Positive arc length between a and b 
    smallest_dist = min(dist, 1-dist) # Can go around the circle in 2 directions 
    return smallest_dist / r  # θ = arc length / radius 

In [4]:
def dispersal(sorted_sample, k, circle):
    """
    Inputs:
        sorted_sample, which is a list of length n with entries that are
           observed variants between 0 and 1, increasing from lowest to highest.
           E.g., if n=4 it could be [0.01,0.1,0.6,0.7].
        k, which is a positive integer 1,2,3,... but no more than n-1.
        circle, which is True if the trait is circular or False if it is linear.
    Output: a vector of length n with each entry i being the 'dispersal
        measure' for the point that was at entry i in sorted_sample. The
        dispersal measure for a point is lower if it is closer to other points
        and higher if it is farther from other points. There are k 'other points'
        considered for each point.
    """
    n = len(sorted_sample)
    if not circle:
        results = []
        for i in range(n):
            pt = sorted_sample[i]  # A given point of interest
            subset_min = max(0, i - k)  # Consider k points below it in the list, unless they'd go below 0
            subset_max = min(n - 1, i + k)  # And k points above it, unless that'd be above the max entry
            sample_subset = sorted_sample[subset_min:subset_max + 1]  # Add +1 to include the last entry
            sample_subset.remove(pt) # Do not want the point of interest in here
            dists = [abs(j - pt) for j in sample_subset]  # Distances of other points to pt
            dists.sort()
            dists = dists[:k]  # Take the k smallest distances
            results.append(sum(dists))
        return results
    elif circle:
        results = []
        for i in range(n):
            pt = sorted_sample[i]  # A given point of interest
            angles = []
            for j in range(n): # Get angles to all other points
                if j != i:  # Do not want angle to self, which will be 0
                    other_pt = sorted_sample[j]
                    angle = get_angle(pt, other_pt)
                    angles.append(angle)
            angles.sort()
            angles = angles[:k]  # Take the k smallest angles
            results.append(sum(angles))
        return results

In [5]:
def prob_choose(x, d, k, pr, circle):
    """
    Input:
        x: same as sorted_sample in def dispersal.
        d: the conformity coefficient d_k(x) in Eq. (1) of the paper.
            If d = "min" then use a conformity coefficient near the lower bound;
            If d = "max" then use a conformity coefficient near the upper bound.
            (Note that this function does not apply to the strong anti-conformity model.) 
        k: same as k in def dispersal.
        pr: precision, used for rounding.
        circle: True if the trait is circular or False if it is linear.
    Output: a list of length n where each entry i corresponds to P(x_i | x) in
        Eq. (1) of the paper. 
    """
    n = len(x)
    if type(d) != str:
        if round(d, pr) == 0:  # Cannot round if it is a string, so had to do previous check
            return [1 / n for j in x]
    
    dispersals = dispersal(x, k, circle)  # Note that x is already sorted 
    if all(round(j,pr) == round(dispersals[0],pr) for j in dispersals): # Then all g's = 0
        return [1 / n for j in dispersals]
    
    avg_dispersal = sum(dispersals) / len(dispersals)
    possible_lbs = [] # Possible lower bounds
    possible_ubs = [] # Possible upper bounds
    g_list = []
    denom_I_lst = [l for l in dispersals if round(l, pr) > round(avg_dispersal, pr)]
    denom_I = sum(denom_I_lst)
    num_zero_lst = [1 for l in dispersals if round(l, pr) == 0]
    num_zero = sum(num_zero_lst) # Note that sum([]) is 0 
    if num_zero == 0: # None of the dispersals equals 0  
        denom_II_lst = [1/l for l in dispersals if round(l, pr) < round(avg_dispersal, pr)]
        denom_II = sum(denom_II_lst)
    
    for j in dispersals:
        if round(j, pr) == round(avg_dispersal, pr): # Equal to average dispersal 
            g_i = 0            
        elif round(j, pr) > round(avg_dispersal, pr): # Above average dispersal 
            g_i = -j/denom_I
        elif round(j, pr) < round(avg_dispersal, pr): # Below average dispersal
            if num_zero > 0:
                if round(j, pr) == 0:
                    g_i = 1/num_zero
                else: 
                    g_i = 0
            elif num_zero == 0:
                g_i = (1/j)/denom_II 
                
        g_list.append(g_i)
        if g_i > 0:
            possible_lbs.append(-1/g_i)
            possible_ubs.append((n-1)/g_i)
        elif g_i < 0:
            possible_lbs.append((n-1)/g_i)
            possible_ubs.append(-1/g_i)
            
    if d == "min": d = max(possible_lbs) + 1e-5 
    elif d == "max": d = min(possible_ubs) - 1e-5
    return [1/n + g*d/n for g in g_list]

In [6]:
def choice(x, d, k, pr, σ, discrete_values, circle, res):
    """
    Inputs: x, d, k, pr: same as in def prob_choose.
            σ: standard deviation.
            discrete_values: either "NA" for continuous case, or a list for discrete case.
            circle: either True or False, depending on whether the trait is a circle.
            res: ignored unless d is "super". 
    Output: the variant that an observing individual acquires, between 0 and 1.
    """
    if d == "super":
        flip = True 
        variants, ptotal = get_variants_ptotal(x, σ, res, flip, circle)
        cts_choice = np.random.choice(variants, size=None, p=ptotal)
    else:
        if d == 0: # Random copying
            μ = random.choice(x)
        else:
            probabilities = prob_choose(x, d, k, pr, circle) 
            probabilities = rounding(probabilities)
            μ = random.choices(x, probabilities)[0]
        
        cts_choice = np.random.normal(loc=μ, scale=σ, size=None) 
        if not circle: # Rejection sampling 
            while cts_choice < 0 or cts_choice > 1:
                cts_choice = np.random.normal(loc=μ, scale=σ, size=None)
        elif circle:
            cts_choice = cts_choice % 1  # This function works whether cts_choice is + or -
    
    if discrete_values == "NA":
        return cts_choice
    else:
        return min(discrete_values, key=lambda i: abs(i - cts_choice))


In [7]:
def simulation(N, n, d, k, pr, σ, gen, init_pop, discrete_values, circle, res):
    """
    Inputs: N: the population size
            n: the number of role models
            d, k, pr, σ: same as in def prob_choose
            gen: the number of generations until the simulation stops
            init_pop: all initial variants in the population
            res: either "NA" or a number, depending on whether d is "super".
    """
    pop = init_pop
    for g in range(gen):
        new_pop = []
        for i in range(N): # For each offspring
            role_model_sample = random.sample(pop, n) # Without replacement each time 
            role_model_sample.sort()
            variant = choice(role_model_sample, d, k, pr, σ, discrete_values, circle, res)
            new_pop.append(variant)
        pop = new_pop
    return pop

### Functions specific to the model of strong anti-conformity (or "super" anti-conformity)

In [8]:
def get_variants(min_value, max_value, res, circle):
    """
    Inputs:
        min_value: the minimum value of the cultural trait.
        max_value: the maximum value of the cultural trait.
        res: the resolution, such that cultural variants are separated
        by 1/res. 
        circle: True if the trait is circular or False if it is linear. 
    Output: a list of cultural variants.
    """
    variants = np.linspace(min_value, max_value, res+1)
    if circle: # Then we do not want to include max_value, as it is the same as min_value. 
        variants = variants[:-1]
    return variants

In [9]:
def get_variants_ptotal(x, σ, res, flip, circle):
    """
    Inputs:
        x: a list of length n with entries that are observed variants 
        between 0 and 1, increasing from lowest to highest.
        E.g., if n=4 it could be [0.01,0.1,0.6,0.7].
        σ: standard deviation of the normal distributions. 
        res: resolution, such that variants are separated by 1/res. 
        flip: True if strong anti-conformity model; False if approximation 
        to random copying model. 
        circle: True if the cultural trait is circular or False if it is linear. 
    """
    variants = get_variants(0.0, 1.0, res, circle)
    ptotal = np.zeros(len(variants))
    for μ in x:
        if not (μ>=0.0 and μ<=1.0):
            print("μ:", μ)
        p = norm.pdf(variants, μ, σ) # The PDF y-axis value for each variant
        
        if circle: 
            # circle_error is how much probability is in the left and right tails outside -1.0 to 2.0 
            circle_error = norm.cdf(-1.0, μ, σ) + (1.0-norm.cdf(2.0, μ, σ))
            
            # Exception if there is too high of an error for this σ. Note that in our study, σ = 0.15 or less 
            assert circle_error <= 1e-10, f"circle_error:{circle_error:.3f} too large with σ:{σ}"
            
            variants_lower = get_variants(-1.0, 0.0, res, circle)
            p_lower = norm.pdf(variants_lower, μ, σ)
            p += p_lower # Add left side (left of 0.0)
            
            variants_higher = get_variants(1.0, 2.0, res, circle)
            p_higher = norm.pdf(variants_higher, μ, σ)
            p += p_higher # Add right side (right of 1.0)
        
        p = p/sum(p)
        ptotal += p
    ptotal = ptotal/sum(ptotal)
    
    if flip: 
        maxp = np.max(ptotal) + 0.0005
        ptotal = -ptotal + maxp
        ptotal = ptotal/sum(ptotal)
    
    assert not np.any(ptotal<0) # Shouldn't have any probabilities<0
    assert np.isclose(sum(ptotal), 1.0) # Should sum to 1.0
    return variants, ptotal


### Filename functions

In [10]:
def circle_name(circle):
    if circle:
        return "_cirT_"
    else:
        return "_cirF_"

In [11]:
def discrete_name(discrete_values):
    if discrete_values == "NA":
        return "discF_"
    else:
        return "discT_"

In [12]:
def conformity_name(d):
    if d == "max":
        return "_dmax"
    elif d == "min":
        return "_dmin"
    elif d == "super":
        return "_dsuper"
    elif d > 0:
        number = str(d).replace('.', '')
        return "_dplus" + number
    elif d < 0:
        number = str(d).replace('.', '')
        return "_dminus" + number
    elif d == 0:
        return "_d0"

In [13]:
def sigma_name(σ):
    number = str(σ).replace('.', '')
    return "_sig" + number

In [14]:
def result_file_name(circle, discrete_values, gen, N, n, k, d, σ, rep_num, distribution):
    return ('jun23_' + distribution + circle_name(circle) + discrete_name(discrete_values)
            + 'gen' + str(gen) + '_N'+str(N)+'_n'+str(n)+'_k'+str(k) 
            + conformity_name(d) + sigma_name(σ) + '_rep' + str(rep_num) +'.csv')

In [15]:
def init_pop_file_name(discrete_values, N, distribution): 
    return ('jun23_init_pop_' + distribution + '_' 
            + discrete_name(discrete_values) + 'N' + str(N) + '.csv')

## Running the simulation

### (a) Simulation with $k=2$ (as in Figure 4), as well as $k=3$ (as in Figure S7.2)

In [ ]:
start_time = time.time()
pr = 9
N = 10000
gen = 100
distribution = "uniform"
circle = False
discrete_values = "NA"
n = 5 

d = "super"

res = "NA"
if d == "super":
    res = 1000

init_pop = pd.read_csv(init_pop_file_name(discrete_values, N, distribution),index_col=0)
init_pop = init_pop['Data'].tolist()

for k in [2,3]:
    for σ in [0.01, 0.05, 0.1, 0.15]:  
        for rep_num in range(1,6):
            result = simulation(N, n, d, k, pr, σ, gen, init_pop, discrete_values, circle, res)
            filename = result_file_name(circle, discrete_values, gen, N, n, k, d, σ, rep_num, distribution)
            print(filename)
            result_ = pd.DataFrame(result)
            result_.columns = ['Data']
            result_.to_csv(filename)
        
print((time.time() - start_time)/60, "minutes")


### (b) Extending to 1000 generations (as in Figure S7.1)

In [ ]:
start_time = time.time() 
pr = 9
N = 10000
k = 2
past_gen = 100
tot_gen = 1000
distribution = "uniform"
circle = False
discrete_values = "NA"
n = 5 

d = "super"

res = "NA"
if d == "super":
    res = 1000

for σ in [0.01, 0.05, 0.1, 0.15]:  
    for rep_num in range(1,6):
        init_file_name = result_file_name(circle, discrete_values, past_gen, N, n, k, d, σ, rep_num, distribution)
        init_pop = pd.read_csv(init_file_name,index_col=0)
        init_pop = init_pop['Data'].tolist()
        print("Starting point:", init_file_name)

        result = simulation(N, n, d, k, pr, σ, tot_gen-past_gen, init_pop, discrete_values, circle, res)
        filename = result_file_name(circle, discrete_values, tot_gen, N, n, k, d, σ, rep_num, distribution)
        print(filename)
        result_ = pd.DataFrame(result)
        result_.columns = ['Data']
        result_.to_csv(filename)

print((time.time() - start_time)/60, "minutes")


### (c) Circular trait model (as in Figure S7.5)

In [ ]:
start_time = time.time()
pr = 9
N = 10000
gen = 100
k = 2
distribution = "uniform"
circle = True
discrete_values = "NA"
n = 5 

d = "super"

res = "NA"
if d == "super":
    res = 1000

init_pop = pd.read_csv(init_pop_file_name(discrete_values, N, distribution),index_col=0)
init_pop = init_pop['Data'].tolist()
print("Init pop:", init_pop_file_name(discrete_values, N, distribution))

for σ in [0.01, 0.05, 0.1, 0.15]:  
    for rep_num in range(1,6):
        result = simulation(N, n, d, k, pr, σ, gen, init_pop, discrete_values, circle, res)
        filename = result_file_name(circle, discrete_values, gen, N, n, k, d, σ, rep_num, distribution)
        print(filename)
        result_ = pd.DataFrame(result)
        result_.columns = ['Data']
        result_.to_csv(filename)

print((time.time() - start_time)/60, "minutes")


### (d) Discrete values (as in Figure 5)

In [ ]:
start_time = time.time()
pr = 9
discrete_values = list(np.arange(0,1,0.05))
discrete_values = [round(i,pr) for i in discrete_values]
discrete_values.append(1.0)
N = 10000
gen = 100
k = 2
distribution = "uniform"
circle = False
#discrete_values = "NA"
n = 5 

d = "super"

res = "NA"
if d == "super":
    res = 1000

init_pop = pd.read_csv(init_pop_file_name(discrete_values, N, distribution),index_col=0)
init_pop = init_pop['Data'].tolist()
print("Init pop:", init_pop_file_name(discrete_values, N, distribution))

for σ in [0.01, 0.05, 0.1, 0.15]:  
    for rep_num in range(1,6):
        result = simulation(N, n, d, k, pr, σ, gen, init_pop, discrete_values, circle, res)
        filename = result_file_name(circle, discrete_values, gen, N, n, k, d, σ, rep_num, distribution)
        print(filename)
        result_ = pd.DataFrame(result)
        result_.columns = ['Data']
        result_.to_csv(filename)

print((time.time() - start_time)/60, "minutes")


### (e) Different population sizes (as in Figure 6)

In [ ]:
start_time = time.time()
pr = 9
gen = 100
k = 2
distribution = "uniform"
circle = False
discrete_values = "NA"
n = 5 
σ = 0.05

d = "super"

res = "NA"
if d == "super":
    res = 1000

for N in [100, 1000, 10000, 20000]:
    init_pop = pd.read_csv(init_pop_file_name(discrete_values, N, distribution),index_col=0)
    init_pop = init_pop['Data'].tolist()
    print("Init pop:", init_pop_file_name(discrete_values, N, distribution))
    
    for rep_num in range(1,6):
        result = simulation(N, n, d, k, pr, σ, gen, init_pop, discrete_values, circle, res)
        filename = result_file_name(circle, discrete_values, gen, N, n, k, d, σ, rep_num, distribution)
        print(filename)
        result_ = pd.DataFrame(result)
        result_.columns = ['Data']
        result_.to_csv(filename)

print((time.time() - start_time)/60, "minutes")


### (f) Different initial population distributions (as in Figure 7)

In [ ]:
start_time = time.time()
pr = 9
gen = 100
N = 10000
k = 2
circle = False
discrete_values = "NA"
n = 5 
σ = 0.05

d = "super"

res = "NA"
if d == "super":
    res = 1000

for distribution in ['leftbeta', 'middlebeta', 'doublebeta', 'rightbeta']:
    init_pop = pd.read_csv(init_pop_file_name(discrete_values, N, distribution),index_col=0)
    init_pop = init_pop['Data'].tolist()
    print("Init pop:", init_pop_file_name(discrete_values, N, distribution))
    
    for rep_num in range(1,6):
        result = simulation(N, n, d, k, pr, σ, gen, init_pop, discrete_values, circle, res)
        filename = result_file_name(circle, discrete_values, gen, N, n, k, d, σ, rep_num, distribution)
        print(filename)
        result_ = pd.DataFrame(result)
        result_.columns = ['Data']
        result_.to_csv(filename)

print((time.time() - start_time)/60, "minutes")
